<a href="https://colab.research.google.com/github/griffithpoindexter/PageRank-HITS-SimRank/blob/master/CIS531_731_Fall_HW2_Final_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
####### CHANGES MADE BY INSTRUCTOR" ########
!pip install -q pyspark
!wget -q https://www.gutenberg.org/files/66057/66057-0.txt

     |████████████████████████████████| 212.4 MB 64 kB/s 
     |████████████████████████████████| 198 kB 61.8 MB/s 


In [2]:
#import appropriate libraries and create Spark and SQL session 
import os
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
# create removePunctuation function to clean data 
import re
import string
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    return regex.sub('', text).lower().strip()
    
print (removePunctuation('YŪGAO'))
print (removePunctuation(' No under_score!'))

yūgao
no underscore


In [68]:
# create removePunctuation function that does not lower case items, removes numbers, and removes roman numerals 
import re
import string
def removePunctuationupper(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    replace_dashes = text.replace("-"," ")
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    data = regex.sub('', replace_dashes).strip()
    remove_numbers = re.compile('[0-9]')
    data_no_numbers = remove_numbers.sub(' ',data)
    remove_numerals = re.compile(r'(?:^|\s)(IX|IV|V?I{0,3})(?:\s|$)')
    return remove_numerals.sub(' ',data_no_numbers)

#Notice the difference between the 
print (removePunctuationupper('YŪGAO12'))
print (removePunctuationupper('Lady-in-Perpetual-Attendance'))

YŪGAO 
Lady in Perpetual Attendance


In [69]:
#create function to remove additional types of punctuation that may have been missed by the removePunctuation function 
import sys
import unicodedata
import re

punc_list=[]

for i in range(0, sys.maxunicode + 1):
  if unicodedata.category(chr(i)) in ('Pc', 'Pd', 'Pe', 'Pf', 'Pi', 'Po', 'Ps'):
    punc_list.append(chr(i))

punc_string=''.join(punc_list)

def removeUniPunctuation(text):
    regex = re.compile('[%s]' % re.escape(punc_string))
    return regex.sub(' ', text).strip()

In [70]:
#import files and parse to 10 different mappers 
from pyspark.sql.functions import regexp_extract, col

fileName = sc.textFile('/content/sample_data/66057-0.txt',10)

remove_numerals = re.compile(r'(?:^|\s)(IX|IV|V?I{0,3})(?:\s|$)')

#first attempt to remove punctuation from data 
fileName2 = sc.textFile('/content/sample_data/66057-0.txt',10).map(lambda x: removeUniPunctuation(x))

#split all words into their own location using .split 
all_words_split = fileName.flatMap(lambda x: x.split())


#search for uppercase letters using regex, map roman number data cleaning logic to strings if the length of the value is > 1, split data into individual cells and count occurrences  
total_uppercase = fileName2.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count()

#search for lowercase letters using regex, map roman number data cleaning logic to strings if the length of the value is > 1, split data into individual cells and count occurrences  
total_lowercase = fileName2.flatMap(lambda x: re.findall(r'\b[a-z]+(?:\s+[a-z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count()

print('total number of uppercase words: ', total_uppercase)
print('total number of lowercase words: ', total_lowercase)

total number of uppercase words:  1403
total number of lowercase words:  89356


In [71]:
#total count of all words in filename - 1. filename2 has logic that incorrectly expands the total word count, so it cannot be used. 
total_word_count  = all_words_split.count()
total_word_count

101277

In [72]:
#read data accross 10 mappers 
#join string into one single line of text 
#split texts based on CHAPTER # and Appendix 3 

import re

book = sc.textFile('/content/sample_data/66057-0.txt',10).collect()

book_single_string = ' '.join(book)

chapters = re.split('CHAPTER \w{1,4} | APPENDIX \w{1}',book_single_string)


In [76]:
#parralize chapter data into their own RDD for analysis 
intro = sc.parallelize([chapters[0]])
chapter1 = sc.parallelize([chapters[1]])
chapter2 = sc.parallelize([chapters[2]])
chapter3 = sc.parallelize([chapters[3]])
chapter4 = sc.parallelize([chapters[4]])
chapter5 = sc.parallelize([chapters[5]])
chapter6 = sc.parallelize([chapters[6]])
chapter7 = sc.parallelize([chapters[7]])
chapter8 = sc.parallelize([chapters[8]])
chapter9 = sc.parallelize([chapters[9]])
appendix = sc.parallelize([chapters[10] + chapters[11]])

#run punctuation logic and place word values into their own lists location 
intro_rdd_word_count = intro.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter1_rdd_word_count = chapter1.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter2_rdd_word_count = chapter2.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter3_rdd_word_count = chapter3.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter4_rdd_word_count = chapter4.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter5_rdd_word_count = chapter5.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter6_rdd_word_count = chapter6.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter7_rdd_word_count = chapter7.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter8_rdd_word_count = chapter8.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter9_rdd_word_count = chapter9.map(removeUniPunctuation).flatMap(lambda x: x.split())
appendix_rdd_word_count = appendix.map(removeUniPunctuation).flatMap(lambda x: x.split())


#percentage of uppercase values by introduction, chapter, appendix 
print('percentage of uppercase values in introduction: ', round(intro_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 1: ',round(chapter1_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 2: ',round(chapter2_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 3: ',round(chapter3_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 4: ',round(chapter4_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 5: ',round(chapter5_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 6: ',round(chapter6_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 7: ',round(chapter7_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 8: ',round(chapter8_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 9: ',round(chapter9_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in appendix: ',round(appendix_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))



percentage of uppercase values in introduction:  6.27
percentage of uppercase values in chapter 1:  4.35
percentage of uppercase values in chapter 2:  18.96
percentage of uppercase values in chapter 3:  2.71
percentage of uppercase values in chapter 4:  13.11
percentage of uppercase values in chapter 5:  16.18
percentage of uppercase values in chapter 6:  7.7
percentage of uppercase values in chapter 7:  4.21
percentage of uppercase values in chapter 8:  2.14
percentage of uppercase values in chapter 9:  8.55
percentage of uppercase values in appendix:  16.11


In [77]:
#import schema related functions 
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.functions import lower, col

#look for these words in the dataframe 
word_search = ['Aoi','Emperor','Genji','Kiritsubo', 'Kōkiden','Koremitsu']

#create datafrme of wordsearch values 
df_word_search = sqlContext.createDataFrame(word_search, StringType())

#create data frame with remove punctuation - upper function 
dfintro = sqlContext.createDataFrame(intro_rdd_word_count.map(removePunctuationupper), StringType())
df1 = sqlContext.createDataFrame(chapter1_rdd_word_count.map(removePunctuationupper), StringType())
df2 = sqlContext.createDataFrame(chapter2_rdd_word_count.map(removePunctuationupper), StringType())
df3 = sqlContext.createDataFrame(chapter3_rdd_word_count.map(removePunctuationupper), StringType())
df4 = sqlContext.createDataFrame(chapter4_rdd_word_count.map(removePunctuationupper), StringType())
df5 = sqlContext.createDataFrame(chapter5_rdd_word_count.map(removePunctuationupper), StringType())
df6 = sqlContext.createDataFrame(chapter6_rdd_word_count.map(removePunctuationupper), StringType())
df7 = sqlContext.createDataFrame(chapter7_rdd_word_count.map(removePunctuationupper), StringType())
df8 = sqlContext.createDataFrame(chapter8_rdd_word_count.map(removePunctuationupper), StringType())
df9 = sqlContext.createDataFrame(chapter9_rdd_word_count.map(removePunctuationupper), StringType())
dfappendix = sqlContext.createDataFrame(appendix_rdd_word_count.map(removePunctuationupper), StringType())

#count the occurends of the word search values by chapter 
print('introduction count: ',dfintro[dfintro['value'].isin(word_search)].count())
print('chapter 1 count: ', df1[df1['value'].isin(word_search)].count())
print('chapter 2 count: ',df2[df2['value'].isin(word_search)].count())
print('chapter 3 count: ',df3[df3['value'].isin(word_search)].count())
print('chapter 4 count: ',df4[df4['value'].isin(word_search)].count())
print('chapter 5 count: ',df5[df5['value'].isin(word_search)].count())
print('chapter 6 count: ',df6[df6['value'].isin(word_search)].count())
print('chapter 7 count: ',df7[df7['value'].isin(word_search)].count())
print('chapter 8 count: ',df8[df8['value'].isin(word_search)].count())
print('chapter 9 count: ',df9[df9['value'].isin(word_search)].count())
print('appendix count: ',dfappendix[dfappendix['value'].isin(word_search)].count())


introduction count:  46
chapter 1 count:  83
chapter 2 count:  93
chapter 3 count:  30
chapter 4 count:  155
chapter 5 count:  156
chapter 6 count:  85
chapter 7 count:  131
chapter 8 count:  50
chapter 9 count:  189
appendix count:  14


In [78]:
#read in file name and run removePunctuation - upper to clean data 
filename_with_data_cleaned_when_create = fileName.map(removePunctuationupper)

#split word values into seperate list locations 
file_name_prep_for_unique_count = filename_with_data_cleaned_when_create.flatMap(lambda x: x.split())

#count words, reduce by key values and count them to obtain unique word values 
unique_count_of_words = file_name_prep_for_unique_count.map(lambda word: (word,1)).reduceByKey(lambda a,b: a+b).count()

print('number of unique words: ', unique_count_of_words)



number of unique words:  9379
